# Pràctica 4

In [1]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/catalan_general_crawling contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/catalan_general_crawling
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 1016113 examples [00:38, 26628.24 examples/s]


In [6]:
dataset = dataset['train']

# i want to see the first 10 examples
for i in range(10):
    print(dataset[i])
    print()

{'text': 'Reduïu els costos dels processos administratius al vostre organisme públic\nEviteu els desplaçaments i pèrdua de temps als ciutadans en les seves gestions\nOferiu una administració més transparent a ciutadans i empreses\nEns grans i petits experimenten aquesta transformació amb èxit, gràcies al suport de l\'AOC\nDepartament de Sistemes d\'Informació i Processos\n" Via Oberta ens ha permès fer efectiu el dret dels ciutadans a no aportar documents, eliminant paper i simplificant procediments"\n" e.FACT proporciona informació indispensable per a la realització de les auditories del registre comptable de factures de les Administracions Públiques Catalanes"\nCoordinador del departament d\'Informàtica\n"El servei VIA OBERTA és el que ha aportat majors avantatges per als ciutadans"\n"Amb l\' e-NOTUM hem escurçat els procediments en 12 dies, quasi un 40% menys!"\nCoordinadora d\'organització de persones i e-administració\n" Via Oberta ofereix millores per als ciutadans al no haver d\

Obtenim diferents mides de datasets

In [18]:
# i want to get the first 100 MB of the dataset
size = 0
for i in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 100000000:
        print(i)
        break

dataset_100MB = dataset.select(list(range(i)))


# i want to get the first 500 MB of the dataset
size = 0
for j in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 500000000:
        print(j)
        break

dataset_500MB = dataset.select(list(range(j)))


# i want to get the first 1 GB of the dataset
size = 0
for g in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 1000000000:
        print(g)
        break

dataset_1GB = dataset.select(list(range(g)))

57333
369003
738007


## Preprocessament

In [32]:
import gensim
from gensim.models import Word2Vec
import re

# Funció per netejar i tokenitzar el text
def preprocess_text(text):
    # Neteja el text: treu caràcters no desitjats, converteix a minúscules, etc.
    text = re.sub(r'\W+', ' ', text)  # Substitueix caràcters no alfanumèrics per espais
    text = text.lower()  # Converteix a minúscules
    words = text.split()  # Tokenitza
    return words

# Carrega el fitxer de text i processa'l
corpus_100MB = []
for line in dataset_100MB:
    words = preprocess_text(line['text'])
    if words:  # Assegura't que la línia no està buida
        corpus_100MB.append(words)

# Comprova algunes línies per assegurar-te que tot està bé
print(corpus_100MB[:2])


[['reduïu', 'els', 'costos', 'dels', 'processos', 'administratius', 'al', 'vostre', 'organisme', 'públic', 'eviteu', 'els', 'desplaçaments', 'i', 'pèrdua', 'de', 'temps', 'als', 'ciutadans', 'en', 'les', 'seves', 'gestions', 'oferiu', 'una', 'administració', 'més', 'transparent', 'a', 'ciutadans', 'i', 'empreses', 'ens', 'grans', 'i', 'petits', 'experimenten', 'aquesta', 'transformació', 'amb', 'èxit', 'gràcies', 'al', 'suport', 'de', 'l', 'aoc', 'departament', 'de', 'sistemes', 'd', 'informació', 'i', 'processos', 'via', 'oberta', 'ens', 'ha', 'permès', 'fer', 'efectiu', 'el', 'dret', 'dels', 'ciutadans', 'a', 'no', 'aportar', 'documents', 'eliminant', 'paper', 'i', 'simplificant', 'procediments', 'e', 'fact', 'proporciona', 'informació', 'indispensable', 'per', 'a', 'la', 'realització', 'de', 'les', 'auditories', 'del', 'registre', 'comptable', 'de', 'factures', 'de', 'les', 'administracions', 'públiques', 'catalanes', 'coordinador', 'del', 'departament', 'd', 'informàtica', 'el', 's

## Entrenament model de similitud de Text Semàntic

In [ ]:
from datasets import load_dataset

text_semantic = load_dataset("projecte-aina/sts-ca")

In [33]:
from gensim.models import word2vec
model = word2vec.Word2Vec(corpus_100MB, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
# Obtenir un word-vector

In [35]:
# Exemple: trobar paraules similars
similar_words = model.wv.most_similar('paraula')
print(similar_words)

[('frase', 0.7374141216278076), ('grafia', 0.6719822883605957), ('lletra', 0.628826379776001), ('cançó', 0.595634400844574), ('locució', 0.5825865268707275), ('terminació', 0.5624357461929321), ('idea', 0.5605849027633667), ('versió', 0.5579149723052979), ('conjunció', 0.5572755932807922), ('recomanació', 0.556980311870575)]


In [36]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Defineix funcions per calcular els embeddings de frases
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector

def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [37]:
# Exemple de corpus
corpus = ["Aquest és un text de exemple", "Aquí hi ha un altre text"]

# Prepara els vectors TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [38]:
# Exemple d'ús amb embeddings pre-entrenats (Word2Vec)
from gensim.models import KeyedVectors
import tensorflow as tf

# Carrega el model pre-entrenat (substitueix 'path_to_model' amb la ruta real)
# word2vec_model = KeyedVectors.load_word2vec_format('path_to_model', binary=True)

# Defineix la longitud de les seqüències i la dimensió dels embeddings
input_length = 10
embedding_dim = model.vector_size

# Prepara les dades d'entrenament
X_train1 = np.array([compute_mean_embedding(text.split(), model.wv, embedding_dim) for text in corpus])
X_train2 = np.array([compute_weighted_mean_embedding(text.split(), model.wv, word2tfidf, embedding_dim) for text in corpus])
y_train = np.random.rand(len(corpus), 1)

# Defineix el model de Keras (ajusta les dimensions segons la longitud de les seqüències i la dimensió dels embeddings)
def build_and_compile_model(input_dim, hidden_size=64):
    input_1 = tf.keras.Input(shape=(input_dim,))
    input_2 = tf.keras.Input(shape=(input_dim,))
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)([input_1, input_2])
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')(concatenate)
    output = tf.keras.layers.Dense(1)(hidden)
    
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

# Construir i compilar el model
m = build_and_compile_model(input_dim=embedding_dim)

# Entrenar el model
m.fit([X_train1, X_train2], y_train, epochs=10, batch_size=32)

# Exemple de predicció
input_data1 = compute_mean_embedding("Aquest és un text de exemple".split(), model.wv, embedding_dim).reshape(1, -1)
input_data2 = compute_weighted_mean_embedding("Aquí hi ha un altre text".split(), model.wv, word2tfidf, embedding_dim).reshape(1, -1)

# Predir la similitud
y_pred = m.predict([input_data1, input_data2])
print(y_pred)

Epoch 1/10
1/1 [==============================] - 0s 334ms/step - loss: 0.7557
Epoch 2/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1112
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4268
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5486
Epoch 5/10
1/1 [==============================] - 0s 3ms/step - loss: 0.4800
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 0.3008
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.0543
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2087
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 0.2880
Epoch 10/10
1/1 [==============================] - 0s 55ms/step
[[0.10263795]]


In [39]:
y_train

array([[0.24829134],
       [0.49559584]])

### Implementació de model de regressió de similitud